In [80]:
# Liberty Mutual Group: Property Inspection Prediction

import numpy as np 
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
import pandas as pd

In [81]:
#load train and test 
df  = pd.read_csv('train.csv', index_col=0)
tf  = pd.read_csv('test.csv', index_col=0)

In [82]:
df.head()

,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,
1,1,15,3,2,N,B,N,B,B,D,...,2,37,1,11,6,Y,N,E,2,2
2,4,16,14,5,H,B,N,B,B,C,...,2,22,1,18,5,Y,Y,E,2,1
3,1,10,10,5,N,K,N,B,B,E,...,6,37,2,14,6,Y,Y,E,6,1
4,1,18,18,5,N,K,N,B,B,E,...,2,25,1,1,6,Y,N,C,2,6
5,1,13,19,5,N,H,N,B,B,E,...,1,22,1,2,7,N,N,E,1,1


In [83]:
tf.head()

,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,T1_V10,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,
6,2,13,4,C,A,Y,B,B,D,12,...,2,28,1,22,6,Y,N,E,2,7
7,10,10,7,N,C,Y,B,B,D,8,...,3,28,1,4,3,Y,N,E,5,8
8,9,20,4,N,H,Y,B,B,E,8,...,2,22,1,1,7,N,N,C,6,1
9,11,18,2,N,H,Y,B,B,D,12,...,4,40,1,20,6,Y,N,E,5,5
10,4,5,4,H,K,Y,B,B,E,8,...,2,34,1,11,7,Y,Y,E,2,1


In [84]:
#Drop the un-necessary features to avoid overfitting 
#I have used the feature selection code which is included in github or rar file
tf.drop('T2_V10', axis=1, inplace=True)
tf.drop('T2_V7', axis=1, inplace=True)
tf.drop('T1_V13', axis=1, inplace=True)
tf.drop('T1_V10', axis=1, inplace=True)

In [85]:
tf.shape

(51000, 28)

In [86]:
labels = df.Hazard
df.drop('Hazard', axis=1, inplace=True)

In [87]:
df.drop('T2_V10', axis=1, inplace=True)
df.drop('T2_V7', axis=1, inplace=True)
df.drop('T1_V13', axis=1, inplace=True)
df.drop('T1_V10', axis=1, inplace=True)

In [88]:
df.shape

(50999, 28)

In [89]:
df.head()

,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,T1_V11,...,T2_V4,T2_V5,T2_V6,T2_V8,T2_V9,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,
1,15,3,2,N,B,N,B,B,D,B,...,10,B,2,1,11,Y,N,E,2,2
2,16,14,5,H,B,N,B,B,C,B,...,17,C,2,1,18,Y,Y,E,2,1
3,10,10,5,N,K,N,B,B,E,H,...,13,C,6,2,14,Y,Y,E,6,1
4,18,18,5,N,K,N,B,B,E,H,...,15,A,2,1,1,Y,N,C,2,6
5,13,19,5,N,H,N,B,B,E,H,...,11,B,1,1,2,N,N,E,1,1


In [90]:
columns = df.columns
test_ind = tf.index

In [91]:
print(columns)
print(test_ind)

Index(['T1_V1', 'T1_V2', 'T1_V3', 'T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8',
       'T1_V9', 'T1_V11', 'T1_V12', 'T1_V14', 'T1_V15', 'T1_V16', 'T1_V17',
       'T2_V1', 'T2_V2', 'T2_V3', 'T2_V4', 'T2_V5', 'T2_V6', 'T2_V8', 'T2_V9',
       'T2_V11', 'T2_V12', 'T2_V13', 'T2_V14', 'T2_V15'],
      dtype='object')
Int64Index([     6,      7,      8,      9,     10,     11,     13,     14,
                16,     17,
            ...
            101977, 101979, 101982, 101983, 101985, 101989, 101990, 101995,
            101996, 101997],
           dtype='int64', name='Id', length=51000)


In [92]:
df = np.array(df)
tf = np.array(tf)

In [93]:
print(df)
print(tf)

[[15 3 2 ..., 'E' 2 2]
 [16 14 5 ..., 'E' 2 1]
 [10 10 5 ..., 'E' 6 1]
 ..., 
 [18 7 5 ..., 'E' 1 1]
 [18 17 5 ..., 'C' 2 6]
 [5 15 3 ..., 'E' 5 4]]
[[2 13 4 ..., 'E' 2 7]
 [10 10 7 ..., 'E' 5 8]
 [9 20 4 ..., 'C' 6 1]
 ..., 
 [13 24 3 ..., 'C' 2 4]
 [9 7 5 ..., 'E' 3 1]
 [4 6 2 ..., 'D' 6 5]]


In [94]:
# label encode the categorical variables this is using label encoder but same thing can be done by one hot encoding also
# I have used one hot encoding in gradient boosting algorithm 
for i in range(df.shape[1]):
    if type(df[1,i]) is str:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df[:,i]) + list(tf[:,i]))
        df[:,i] = lbl.transform(df[:,i])
        tf[:,i] = lbl.transform(tf[:,i])

df = df.astype(float)
tf = tf.astype(float)


In [95]:
print(df)
print(tf)

[[ 15.   3.   2. ...,   4.   2.   2.]
 [ 16.  14.   5. ...,   4.   2.   1.]
 [ 10.  10.   5. ...,   4.   6.   1.]
 ..., 
 [ 18.   7.   5. ...,   4.   1.   1.]
 [ 18.  17.   5. ...,   2.   2.   6.]
 [  5.  15.   3. ...,   4.   5.   4.]]
[[  2.  13.   4. ...,   4.   2.   7.]
 [ 10.  10.   7. ...,   4.   5.   8.]
 [  9.  20.   4. ...,   2.   6.   1.]
 ..., 
 [ 13.  24.   3. ...,   2.   2.   4.]
 [  9.   7.   5. ...,   4.   3.   1.]
 [  4.   6.   2. ...,   3.   6.   5.]]


In [96]:
#firstly we can select array of n_estimator values as 50 100 150 but after that we will get that 100 produces best outcome
param_grid = {'n_estimators': [100]}
model = GridSearchCV(RandomForestRegressor(), param_grid)

In [97]:
#Fit the model and produce best CV score
model = model.fit(df,labels)
print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

Best parameters found by grid search:
{'n_estimators': 100}
Best CV score:
0.06405653462843511


In [98]:
#predict the model
#create the pickle file for future use
preds = model.predict(tf)
joblib.dump(model, 'Random_forest_regressor_model.pkl')

['Random_forest_regressor_model.pkl']

In [99]:
#Now predict the test data and save that data to csv file using .to_csv
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('abhishek_jagtap_rf_model.csv')

In [ ]:
#thank you
#program produced the score of 0.343668 whose proof is included in the screenshot attached within the documents
